In [1]:
# %%
import time
import numpy as np
import pickle
from numpy.linalg import det

import CMINE_lib as CMINE
# from Guassian_variables import Data_guassian
from structurerl import * 

import pandas as pd
from scipy.stats import multivariate_normal
import itertools

np.random.seed(37)
from scipy import stats
from sklearn.neighbors import KernelDensity

import math

import torch 
import torch.nn as nn
import torch.nn.functional as F

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'


In [2]:
def log_sum_exp(value, dim=None, keepdim=False):
    """Numerically stable implementation of the operation
    value.exp().sum(dim, keepdim).log()
    """
    # TODO: torch.max(value, dim=None) threw an error at time of writing
    if dim is not None:
        m, _ = torch.max(value, dim=dim, keepdim=True)
        value0 = value - m
        if keepdim is False:
            m = m.squeeze(dim)
        return m + torch.log(torch.sum(torch.exp(value0),
                                       dim=dim, keepdim=keepdim))
    else:
        m = torch.max(value)
        sum_exp = torch.sum(torch.exp(value - m))
        if isinstance(sum_exp, Number):
            return m + math.log(sum_exp)
        else:
            return m + torch.log(sum_exp)

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [4]:
from modules.CMI import DR_CMI, CDL_CMI

In [5]:
Dim = 5
batch_size = 64
#dataset = CMINE.create_dataset_DGP( Dim=5, N=batch_size)
dataset = create_dataset_DGP_binary_A_conf( Dim=5, N=batch_size)
s_t = torch.from_numpy(dataset[0]).float().cuda()
s_next = torch.from_numpy(dataset[1]).float().cuda()
a = torch.from_numpy(dataset[2]).float().cuda()

In [6]:
sample_dim = 2*Dim

hidden_size = 15
learning_rate = 0.005
training_steps = 10

cubic = False 


In [7]:
def train_dr(N = 64, training_steps = 10, noise = 0.1):
    torch.cuda.empty_cache()
    model_dr = DR_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_dr = torch.optim.Adam(model_dr.parameters(), learning_rate)
    dr_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        #dataset = create_dataset_DGP_binary_A_conf(Dim=Dim, N=N)
        dataset = create_dataset_DGP_binary_A_more_noise(Dim=Dim, N=N, noise = noise)
        #dataset = create_dataset_DGP_binary_A(Dim=Dim, N=N)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()
        #print(a)

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_dr.eval()
        drs = model_dr(batch_x, batch_y)
        #mi_est_values.append(cmi)
        dr_est_values.append(drs)
        model_dr.train() 

        model_loss = model_dr.learning_loss(batch_x, batch_y)

        optimizer_dr.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_dr.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return dr_est_values

In [8]:
def train_cdl(N = 64, training_steps = 10, noise = 0.1):
    torch.cuda.empty_cache()
    model_cdl = CDL_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_cdl = torch.optim.Adam(model_cdl.parameters(), learning_rate)
    cdl_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        #dataset = create_dataset_DGP_binary_A_conf(Dim=Dim, N=64)
        dataset = create_dataset_DGP_binary_A_more_noise(Dim=Dim, N=N, noise = noise)
        #dataset = create_dataset_DGP_binary_A(Dim=Dim, N=64)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_cdl.eval()
        cdl_cmi = model_cdl(batch_x, batch_y)
        cdl_est_values.append(cdl_cmi)
        model_cdl.train() 

        model_loss = model_cdl.learning_loss(batch_x, batch_y)

        optimizer_cdl.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_cdl.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return cdl_est_values

In [9]:
N = 8
cdl_est_values = train_cdl(N, 100)
dr_est_values = train_dr(N, 100)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[5.08203380e+03 5.07689674e+03 5.07596928e+03 5.07691093e+03
 5.07763493e+03 5.20894909e+07 2.49430462e+14 2.66375671e+08
 2.91236501e+07 3.24908658e+07]
[5.41008848e+03 5.28923849e+03 4.03628418e+05 5.12845097e+03
 5.51013351e+05 1.49308878e+08 2.57488784e+09 1.21480156e+06
 2.73027260e+11 2.84857181e+06]


In [10]:
N = 16
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[ 0.25160089 20.90705369 41.72471771  0.22760719  0.66024089  0.56176038
  1.67688251  5.35747147  0.70878812  2.89557232]
[1.02449280e+00 6.80109338e-01 8.48897291e-01 1.01883583e+00
 1.62740350e+02 1.33371779e+02 9.96202892e+01 9.51680172e+01
 6.79234699e+01 1.55864088e+04]


In [11]:
N = 32
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[4.70797213e-01 1.03628284e+00 2.52753370e-01 1.80713377e+01
 3.27465548e-01 1.02087174e+01 1.39288551e+02 3.37460004e+01
 6.44902174e+02 5.10537716e+00]
[5.39571622e+03 5.83858509e+03 8.55009451e+03 5.26270402e+03
 5.52547864e+03 1.18610010e+08 9.64424706e+08 5.50517226e+03
 8.65865125e+05 7.54818882e+04]


In [17]:
N=64
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[0.20533267 6.41830869 0.36476142 2.17412648 0.20718515 7.70160287
 0.36651939 9.37775562 0.50441016 2.954366  ]
[3.69959268e+02 6.12342524e+02 5.90525959e+02 6.95779578e+02
 5.83077422e+02 1.96545525e+08 8.87359976e+02 1.33609852e+04
 2.45061627e+05 9.25536269e+04]


In [16]:
N=128
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[0.67332595 0.07067285 0.06221422 0.05769692 0.05641404 0.22059119
 0.31473258 0.43435641 0.49405592 0.14012249]
[4.16078431e+01 1.06862337e+02 1.24928540e+02 1.00113302e+02
 8.76377323e+01 3.94086891e+05 1.73749864e+09 1.10081746e+05
 2.85273925e+04 2.87328206e+07]


# Add Noise

In [20]:
for N in [8, 16, 32, 64, 128]:
    cdl_est_values = train_cdl(N, 50, 0.5)
    dr_est_values = train_dr(N, 50, 0.5)
    print(np.array(cdl_est_values).mean(axis=0))
    print(np.array(dr_est_values).mean(axis=0))
    print("--"*5 + str(N)+"--"*5 )

[1.50807499e+02 4.62607846e+00 4.17793512e+00 4.58887104e+00
 5.30967041e+00 2.43819219e+05 2.32101256e+03 1.81911865e+04
 5.60197254e+02 5.53280915e+02]
[1.38506134e+12 2.51517883e+11 1.84226267e+05 3.20695053e+07
 3.31736433e+06 4.06365373e+09 2.08861528e+07 2.03350160e+05
 2.79993891e+08 2.25176659e+05]
----------8----------
[6.46139278e+06 6.46299686e+06 6.46146165e+06 6.46163432e+06
 6.50703142e+06 1.85626803e+10 3.98990505e+15 3.55471859e+08
 1.49689870e+09 1.91844620e+10]
[4.23173144e+04 2.46368612e+02 5.52164237e+02 5.31464425e+02
 3.09564114e+02 1.22811745e+08 5.11384563e+04 5.11401818e+04
 5.38226931e+04 9.97454350e+05]
----------16----------
[2.00956854e+01 1.55605180e+01 6.28407089e+01 1.71832725e+01
 1.37664207e+01 2.64652491e+07 9.95282069e+02 1.09864795e+03
 1.37166580e+06 2.55753486e+02]
[1.77034982e+03 2.41482696e+03 2.57785120e+04 2.39645013e+03
 1.21346664e+06 1.29198787e+09 3.39466295e+05 1.15756326e+05
 6.47215526e+08 3.21895300e+06]
----------32----------
[ 65.032

In [14]:
N = 8
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[3.64212415e+02 3.63155174e+02 3.62741391e+02 3.63982946e+02
 3.62547022e+02 3.54660821e+06 1.17143885e+05 6.30458812e+03
 2.02804373e+05 6.58121819e+03]
[5.34660669e+06 5.39030692e+06 5.34102615e+06 5.31054044e+06
 5.68449332e+06 5.41413441e+16 8.78900676e+08 1.34109684e+10
 1.45785828e+12 2.63270089e+10]


In [15]:
N = 8
cdl_est_values = train_cdl(N, 100)
dr_est_values = train_dr(N, 100)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[2.22504738e+01 2.67431462e+06 7.73034164e+02 3.96541044e+02
 1.43597268e+02 4.97873293e+04 2.88397809e+03 5.05354645e+05
 7.70204409e+02 7.59108282e+03]
[2.53562779e+04 6.14453626e+06 1.12875001e+05 2.62410940e+03
 1.53140977e+03 3.24888070e+04 4.25532312e+04 8.11987926e+03
 1.10298786e+08 2.56945881e+04]


# Training Step

In [26]:
N = 64
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[6.31943605e+04 1.71830261e+08 6.60621143e+04 1.83854909e+05
 6.29557521e+04 3.95083027e+09 5.30933075e+05 3.03847897e+05
 3.01877772e+06 9.41156254e+05]
[1.45674717e+05 5.62305784e+05 1.31564423e+05 1.34648689e+05
 1.32427251e+06 2.39606343e+09 1.46911331e+14 6.66713691e+09
 1.28531635e+08 2.40902858e+09]


In [27]:
N = 32
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.68653295e+04 8.02705649e+00 1.19907736e+03 3.14780780e+01
 3.98932782e+01 8.38730004e+05 3.92318122e+03 6.95953489e+04
 1.22019822e+05 3.48546747e+06]
[9.74688966e+03 7.74597234e+03 2.96815940e+09 6.31418542e+03
 6.20089638e+03 1.22671743e+06 3.68697284e+09 4.77083438e+06
 1.44713419e+07 1.88002237e+07]


In [31]:
N = 526
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.50006065e+01 2.18871596e+02 1.10105989e+07 5.11114454e+02
 1.06051950e+02 9.00031509e+01 1.04145438e+04 3.52512225e+03
 5.52125692e+03 2.41971664e+04]
[7.68950225e+06 7.74432824e+06 7.91731634e+06 7.70377130e+06
 7.70934032e+06 7.88427070e+08 4.68894041e+16 1.52884448e+10
 1.34091251e+11 2.40433729e+10]


In [30]:
N = 526
training_step = 10
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[2.41623130e-01 5.83663910e-01 2.62432172e-01 1.86141613e-01
 2.88988191e+02 2.00060656e+01 5.57397880e+00 2.69901884e+01
 2.13138771e-01 1.53580540e+00]
[1.34774063e+06 7.49846064e+03 7.87508115e+03 7.57209025e+03
 7.50388969e+03 5.57463735e+05 1.85607135e+04 3.51530271e+09
 1.84042864e+05 2.73516473e+06]
